In [71]:
import pandas as pd
import math

In [53]:
def read_text_document(file_name):
    file = open(file_name, 'rt')
    text = file.read()
    file.close()
    return text

#read files and strip for '.' and '!'
salme1 = (read_text_document("./data/salme_1.txt")).replace('.', '').replace('!', '')
salme2 = (read_text_document("./data/salme_2.txt")).replace('.', '').replace('!', '')
popsang1 = (read_text_document("./data/popsang_1.txt")).replace('.', '').replace('!', '')


In [54]:
# split into words by white space
salme1_words = salme1.split()
salme2_words = salme2.split()
popsang1_words = popsang1.split()


In [55]:
# joined sets of words to remove duplications
salme1_salme2 = set(salme1_words).union(set(salme2_words))
salme1_popsang1 = set(salme1_words).union(set(popsang1_words))
salme2_popsang1 = set(salme2_words).union(set(popsang1_words))

In [56]:
# corpus sizes
print('s1s2_len =', len(salme1_salme2))
print('s1p1_len =', len(salme1_popsang1))
print('s2p1_len =', len(salme2_popsang1))

s1s2_len = 260
s1p1_len = 235
s2p1_len = 267


In [80]:
# create dicts with the words are the keys and the numbers of their appearance as values
#
#Salme_1 vs Salme_2
dict_s1 = dict.fromkeys(salme1_salme2, 0) 
for word in salme1_words:
    dict_s1[word] +=1

dict_s2 = dict.fromkeys(salme1_salme2, 0) 
for word in salme2_words:
    dict_s2[word] +=1


#Salme_1 vs Popsang_1
dict_s1_ = dict.fromkeys(salme1_popsang1, 0) 
for word in salme1_words:
    dict_s1_[word] +=1

dict_p1 = dict.fromkeys(salme1_popsang1, 0) 
for word in popsang1_words:
    dict_p1[word] +=1


#Salme_2 vs Popsang_1
dict_s2_ = dict.fromkeys(salme2_popsang1, 0) 
for word in salme2_words:
    dict_s2[word] +=1

dict_p1_ = dict.fromkeys(salme2_popsang1, 0) 
for word in popsang1_words:
    dict_p1_[word] +=1


## Similarity of Two Vectors

In [72]:
# Calculate dot product of two vectors, divide it by the magnitudes to find the cos(angle between them)
# Use the result as a correlation coefficient 
from collections import Counter

def cosine(vector1, vector2):
     # calculate nominator as a dot product
     intersect = set(vector1.keys()) & set(vector2.keys())
     numerator = sum([vector1[x] * vector2[x] for x in intersect])
    
     # calculate the denominator 
     sum1 = sum([vector1[x] ** 2 for x in list(vector1.keys())])
     sum2 = sum([vector2[x] ** 2 for x in list(vector2.keys())])
    
     denominator = math.sqrt(sum1) * math.sqrt(sum2)
     if not denominator:
         return 0.0
     else:
         return float(numerator)/denominator


In [82]:
# calculate the correlation
corr = cosine(dict_s1, dict_s2)
print("Similarity s1-s2: ", corr)

corr1 = cosine(dict_s1_, dict_p1)
print("Similarity s1-p1: ", corr1)

corr2 = cosine(dict_s2_, dict_p1_)
print("Similarity: s2-p1 ", corr2)


Similarity s1-s2:  0.4996354089336267
Similarity s1-p1:  0.2847018172236098
Similarity: s2-p1  0.0


## Importance of Each Word

In [83]:
# a function to compute the TD frequency of appearance of a word in documents
def computeTF(dicto, doc):
    tfDict = {}
    corpus = len(doc) # number of all words
    for word, wcount in dicto.items():
        tfDict[word] = wcount/float(corpus) # calculete the proportion
    return(tfDict)

In [85]:
# call the function for both sets
tf1 = computeTF(dict_s1, salme1_words)
tf2 = computeTF(dict_s2, salme2_words)

# store into dataframe
tf = pd.DataFrame([tf1, tf2])
tf

,"vælde,",ej,prise,sammen:,vor,sted,Paradis,isbjerg,rent,Forsoner,...,hvor,mange,betalte,"livet,",op,kraft,må,græs,harpestrenge?,rige
0,0.004762,0.004762,0.004762,0.004762,0.004762,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.019048,0.009524,0.000000,0.000000,0.014286,0.004762,0.000000,0.004762,0.004762,0.004762
1,0.000000,0.008299,0.000000,0.000000,0.000000,0.008299,0.008299,0.008299,0.008299,0.008299,...,0.000000,0.000000,0.008299,0.008299,0.000000,0.000000,0.016598,0.000000,0.000000,0.000000


In [88]:
# IDF -  inverse of the document frequency which measures the informativeness of term t
#  lower occurance - higher importance of the word
# idf(t) = N/df
# to avoid div by zero: idf(t) = log(N/(df + 1))

def computeIDF(docList):
    idf = {}
    N = len(docList)
    
    idf = dict.fromkeys(docList[0].keys(), 0)
    for word, wcount in idf.items():
        idf[word] = math.log10(N/(float(wcount) + 1))
        
    return(idf)

#inputing our sentences in the log file
idfs = computeIDF([dict_s1, dict_s2])
idfs

{'vælde,': 0.3010299956639812,
 'ej': 0.3010299956639812,
 'prise': 0.3010299956639812,
 'sammen:': 0.3010299956639812,
 'vor': 0.3010299956639812,
 'sted': 0.3010299956639812,
 'Paradis': 0.3010299956639812,
 'isbjerg': 0.3010299956639812,
 'rent': 0.3010299956639812,
 'Forsoner': 0.3010299956639812,
 'gengælde,': 0.3010299956639812,
 'døden;': 0.3010299956639812,
 'blomsten': 0.3010299956639812,
 'Det': 0.3010299956639812,
 'gør': 0.3010299956639812,
 'Amen': 0.3010299956639812,
 'klipper': 0.3010299956639812,
 'årer,': 0.3010299956639812,
 'han': 0.3010299956639812,
 'værd,': 0.3010299956639812,
 'sammenslår': 0.3010299956639812,
 'fuglesving,': 0.3010299956639812,
 'enge,': 0.3010299956639812,
 'Ræk': 0.3010299956639812,
 'alle': 0.3010299956639812,
 'ser': 0.3010299956639812,
 'giver': 0.3010299956639812,
 'kærlighed?': 0.3010299956639812,
 'i': 0.3010299956639812,
 'tager,': 0.3010299956639812,
 'visdom': 0.3010299956639812,
 'os': 0.3010299956639812,
 'dig,': 0.3010299956639812,

In [89]:
# calculate tf-idf as a measure for the importance of a word
# tf-idf(t, d) = tf(t, d) * log(N/(df + 1))
def computeTFIDF(tf, idfs):
    tfidf = {}
    for word, wcount in tf.items():
        tfidf[word] = wcount*idfs[word]
    return(tfidf)


In [91]:
#running our two sentences through the IDF:
idf1 = computeTFIDF(tf1, idfs)
idf2 = computeTFIDF(tf2, idfs)

# store in a dataframe
idf= pd.DataFrame([idf1, idf2])
idf

,"vælde,",ej,prise,sammen:,vor,sted,Paradis,isbjerg,rent,Forsoner,...,hvor,mange,betalte,"livet,",op,kraft,må,græs,harpestrenge?,rige
0,0.001433,0.001433,0.001433,0.001433,0.001433,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.005734,0.002867,0.000000,0.000000,0.0043,0.001433,0.000000,0.001433,0.001433,0.001433
1,0.000000,0.002498,0.000000,0.000000,0.000000,0.002498,0.002498,0.002498,0.002498,0.002498,...,0.000000,0.000000,0.002498,0.002498,0.0000,0.000000,0.004996,0.000000,0.000000,0.000000
